<a href="https://colab.research.google.com/github/SHREERAJ11/SafeWaters/blob/main/Model/Baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [8]:
import os
import numpy as np
import json
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [9]:
# Load JSON files
with open('/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Questions/Training Question.json') as f:
    train_questions = json.load(f)
with open('/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Questions/Valid Question.json') as f:
    valid_questions = json.load(f)
with open('/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Questions/Test_Question.json') as f:
    test_questions = json.load(f)

questions = {
    'train': train_questions,
    'valid': valid_questions,
    'test': test_questions
}

# Define image folders
image_folders = {
    'train': '/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Train_Image',
    'valid': '/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Valid_Image',
    'test': '/content/gdrive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 2/Images/Test_Image'
}

In [10]:
# Function to preprocess images
def preprocess_images(image_folder, question_data, target_size=(128, 128)):
    image_list = []
    labels = []
    for key, value in question_data.items():
        image_id = value['Image_ID']
        ground_truth = value.get('Ground_Truth')  # Use .get to handle missing keys
        if ground_truth:
            image_path = os.path.join(image_folder, image_id)
            img = cv2.imread(image_path)
            if img is not None:  # Ensure the image is read correctly
                img_resized = cv2.resize(img, target_size)  # Resize image
                image_list.append(img_resized)
                labels.append(ground_truth)
    return np.array(image_list), np.array(labels)

In [ ]:
# Preprocess images and labels for training data
train_images, train_labels = preprocess_images(image_folders['train'], questions['train'])
valid_images, _ = preprocess_images(image_folders['valid'], questions['valid'])
test_images, _ = preprocess_images(image_folders['test'], questions['test'])

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_encoded = to_categorical(train_labels_encoded)

In [ ]:
# Train-validation split
train_images, val_images, train_labels_encoded, val_labels_encoded = train_test_split(
    train_images, train_labels_encoded, test_size=0.2, random_state=42)

In [ ]:
# Print shapes to verify
print(f"Train Images: {train_images.shape}, Train Labels: {train_labels_encoded.shape}")
print(f"Validation Images: {val_images.shape}, Validation Labels: {val_labels_encoded.shape}")
print(f"Test Images: {test_images.shape}")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Print the model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_images, train_labels_encoded,
    epochs=10,
    batch_size=32,
    validation_data=(val_images, val_labels_encoded),
    verbose=2
)

In [ ]:
# Save the model
model.save('cnn_model.h5')

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_images, val_labels_encoded)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

In [ ]:
# Predict on the test data
test_predictions = model.predict(test_images)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_predicted_ground_truth = label_encoder.inverse_transform(test_predicted_labels)

In [ ]:
# Print some predictions
for i in range(5):
    print(f"Test Image {i+1}: Predicted Ground Truth: {test_predicted_ground_truth[i]}")